## Import libraries

In [6]:
import sys, os

basepath = os.path.abspath("../..")
sys.path.append(basepath)
sys.path.append(basepath + "/Operators")
sys.path.append(basepath + "/Utilities")

import numpy as np
import scipy as sp
import time
import matplotlib.pyplot as plt
%matplotlib inline

import DevitoOperators
from DevitoUtils import create_model, plot_image, plot_image_tx, plot_shotrecord
from examples.seismic.acoustic import AcousticWaveSolver
from examples.seismic import plot_velocity, plot_perturbation, demo_model, AcquisitionGeometry

from devito import configuration
configuration['log-level'] = 'WARNING'

from devito import configuration
configuration['log-level'] = 'WARNING'

## Code to run

In [7]:
filestr = "point_scatterers_image_cig"

# Create params dicts
params = {
    "Nx": 300,
    "Nz": 100,
    "Nt": 100,   # this has to be updated later
    "nbl": 75,
    "Ns": 1,
    "Nr": 300,
    "so": 4,
    "to": 2
}

# Create velocity
vel = create_model(shape=(params["Nx"], params["Nz"]))
vel.vp.data[:, :] = 2.0

# Simulation time, wavelet
t0 = 0.
tn = 2000.  # Simulation last 2 second (2000 ms)
f0 = 0.010  # Source peak frequency is 10Hz (0.010 kHz)

# Reflection acquisition geometry (sources and receivers are equally spaced in X direction)
src_depth = 20.0  # Depth is 20m
rec_depth = 20.0  # Depth is 20m

src_coord = np.empty((params["Ns"], 2))
src_coord[:, 0] = np.linspace(0, vel.domain_size[0], num=params["Ns"])
src_coord[:, 1] = src_depth

rec_coord = np.empty((params["Nr"], 2))
rec_coord[:, 0] = np.linspace(0, vel.domain_size[0], num=params["Nr"])
rec_coord[:, 1] = rec_depth

# Create the geometry objects for background velocity models
src_dummy = np.empty((1, 2))

src_dummy[0, :] = src_coord[int(src_coord.shape[0] / 2), :]
geometry = AcquisitionGeometry(vel, rec_coord, src_dummy, t0, tn, f0=f0, src_type='Ricker')
params["Nt"] = geometry.nt
del src_dummy

# Define a solver object
solver = AcousticWaveSolver(vel, geometry, space_order=params["so"])

# Create point scatterers
dm = np.zeros((params["Nt"], params["Nx"], params["Nz"]), dtype=np.float32)
temp = np.zeros((params["Nx"], params["Nz"]), dtype=np.float32)
temp[int(params["Nx"] / 2), int(params["Nz"] * 0.2)] = 1.0
temp[int(params["Nx"] / 2), int(params["Nz"] * 0.4)] = 1.0
temp[int(params["Nx"] / 2), int(params["Nz"] * 0.6)] = 1.0
temp[int(params["Nx"] / 2), int(params["Nz"] * 0.8)] = 1.0
temp = sp.ndimage.gaussian_filter(input=temp, sigma=1, mode="nearest")
for i in range(params["Nt"]):
    dm[i, :, :] = temp

plot_image(dm[0, :, :], colorbar=False, colormap="Greys")

AttributeError: 'numpy.ndarray' object has no attribute 'domain_size'